In [1]:
import numpy as np
import pandas as pd

In [2]:
anime=pd.read_csv("anime.csv")
ratings=pd.read_csv("rating_complete.csv")

In [3]:
ratings.rename(columns={"anime_id":"MAL_ID"},inplace=True)

In [4]:
anime=anime[["MAL_ID","Name"]]

In [5]:
anime.head()

,MAL_ID,Name
0,1,Cowboy Bebop
1,5,Cowboy Bebop: Tengoku no Tobira
2,6,Trigun
3,7,Witch Hunter Robin
4,8,Bouken Ou Beet


In [6]:
anime_with_ratings=ratings.merge(anime,on="MAL_ID")

In [7]:
anime_with_ratings

,user_id,MAL_ID,rating,Name
0,0,430,9,Fullmetal Alchemist: The Conqueror of Shamballa
1,0,1004,5,Kanojo to Kanojo no Neko
2,0,3010,7,Kaiketsu Zorro
3,0,570,7,Jin-Rou
4,0,2762,9,Igano Kabamaru
...,...,...,...,...
57633273,353404,502,8,Dragon Ball Movie 1: Shen Long no Densetsu
57633274,353404,987,4,Dragon Ball GT: Gokuu Gaiden! Yuuki no Akashi ...
57633275,353404,225,8,Dragon Ball GT
57633276,353404,243,7,Gravitation


In [8]:
rating_num=anime_with_ratings.groupby("Name").count()["rating"].reset_index() #find num of rating on each book
rating_num.rename(columns={"rating":"num_of_ratings"},inplace=True)
rating_num

,Name,num_of_ratings
0,"""0""",941
1,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...",18
2,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",4739
3,"""Bungaku Shoujo"" Memoire",5154
4,"""Bungaku Shoujo"" Movie",9718
...,...,...
16865,xxxHOLiC Rou,8269
16866,xxxHOLiC Shunmuki,10373
16867,Üks Uks,11
16868,ēlDLIVE,3330


In [9]:
rating_num["num_of_ratings"].describe()

count     16870.000000
mean       3416.317605
std       10304.755236
min           1.000000
25%          40.000000
50%         266.000000
75%        1671.750000
max      182375.000000
Name: num_of_ratings, dtype: float64

In [10]:
# avg_ratings=anime_with_ratings.groupby("Name",as_index=False)["rating"].mean()
# avg_ratings.rename(columns={"rating":"avg_rating"},inplace=True)
# avg_ratings

In [11]:
x=anime_with_ratings.groupby("user_id").count()["rating"]>200
know_viewers=x[x].index
filtered_ratings=anime_with_ratings[anime_with_ratings["user_id"].isin(know_viewers)]
filtered_ratings

,user_id,MAL_ID,rating,Name
189,3,25835,8,Shirobako
190,3,28171,8,Shokugeki no Souma
191,3,32282,8,Shokugeki no Souma: Ni no Sara
192,3,35788,9,Shokugeki no Souma: San no Sara
193,3,9253,9,Steins;Gate
...,...,...,...,...
57633028,353398,33421,6,Hitori no Shita: The Outcast
57633029,353398,37007,4,Hinomaruzumou
57633030,353398,28825,6,Himouto! Umaru-chan
57633031,353398,34028,3,Idol Jihen


In [12]:
y=filtered_ratings.groupby('Name').count()["rating"]>=2000
famous_anime=y[y].index
final_ratings=filtered_ratings[filtered_ratings["Name"].isin(famous_anime)]
final_ratings

,user_id,MAL_ID,rating,Name
189,3,25835,8,Shirobako
190,3,28171,8,Shokugeki no Souma
191,3,32282,8,Shokugeki no Souma: Ni no Sara
192,3,35788,9,Shokugeki no Souma: San no Sara
193,3,9253,9,Steins;Gate
...,...,...,...,...
57633027,353398,37614,7,Hitoribocchi no Marumaru Seikatsu
57633028,353398,33421,6,Hitori no Shita: The Outcast
57633029,353398,37007,4,Hinomaruzumou
57633030,353398,28825,6,Himouto! Umaru-chan


In [13]:
pt=final_ratings.pivot_table(index="Name",columns="user_id",values="rating")

MemoryError: Unable to allocate 2.56 GiB for an array with shape (94622, 3626) and data type float64

In [ ]:
pt.fillna(0,inplace=True)
pt.head(6)

In [ ]:
pt.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores=cosine_similarity(pt)
similarity_scores.shape

In [ ]:
similarity_scores

In [ ]:
def recommend(anime_name):
    #fetch index
    index=np.where(pt.index==anime_name)[0][0]
    similar_items=sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True) [1:6]

    for i in similar_items:
        print(pt.index[i[0]])

In [ ]:
recommend("Dragon Ball")

In [ ]:
final_ratings[15]